In [1]:
# Standard packages
import tempfile
import warnings
import urllib
import shutil
import os
# Less standard, but still pip- or conda-installable
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
#import rasterio
import re
import rtree
import shapely
import pickle
import data_eng.az_proc as ap
import data_eng.form_calcs as fc

#from cartopy import crs
import collections
import cv2
import math
from glob import glob
from tqdm.notebook import tqdm_notebook

In [2]:
#The file paths of .jpgs in the entire storage space
parent_directory = "//oit-nas-fe13dc.oit.duke.edu//data_commons-borsuk//"

path_storage_space = os.path.join(parent_directory)
jpgs_storage_space = glob(path_storage_space + "/**/*.jpg", recursive = True)
print(len(jpgs_storage_space))

tile_names_tile_urls_complete_array_unique_standard_tile_names = np.load("tile_names_tile_urls_complete_array_unique_standard_tile_names.npy")

#The file names (with and without extensions) for all jpgs in the storage space 
#jpgs_ext, jpgs_without_ext = fc.jpg_path_to_jpg_name_formatted(jpgs_storage_space)

2293088


In [3]:
#Determine the corresponding tile names (formatted) for given paths of jpgs 
tile_names_formatted_storage_space = fc.jpg_path_to_tile_name_formatted(jpgs_storage_space)
tile_names_formatted_storage_space = np.unique(tile_names_formatted_storage_space)
tile_names_formatted_storage_space.shape

#Deteremine the tile names (standard) for given paths of jpgs 
tile_names_standard_storage_space = []
for tile_name_formatted_storage_space in tile_names_formatted_storage_space:
    tile_names_standard_storage_space.append(tile_name_formatted_storage_space.split("_",4)[4])
    
#Create a 2d array of the standard and formatted tile names of the jpgs given
tile_names_in_storage_space_standard_formatted = np.hstack((np.array(tile_names_standard_storage_space)[:,None], tile_names_formatted_storage_space[:,None]))

In [4]:
counter_0 = 0
counter_1 = 0
counter_gr1 = 0
for tile_name_standard in tile_names_in_storage_space_standard_formatted[:,0]: #iterate over the tiles that have actually been annotated
    strings_with_substring = [string for string in tile_names_tile_urls_complete_array_unique_standard_tile_names[:,2] if tile_name_standard in string] #check if a given tile is in the complete_array 
    if len(strings_with_substring) == 0:
        counter_0 += 1
        print(tile_name_standard)
    if len(strings_with_substring) == 1:
        counter_1 += 1
    if len(strings_with_substring) > 1:
        counter_gr1 += 1
print(counter_0, counter_1, counter_gr1)

0 2437 0


In [ ]:
# Check which standard tile names that could be included in the dataset are missing + which tiles have been annotated in duplicate

In [5]:
counter_0 = 0
counter_1 = 0
counter_gr1 = 0
unlabeled_array = []
duplicated_standard_tile_names = []
for tile_name_standard_complete_array in tile_names_tile_urls_complete_array_unique_standard_tile_names: #iterate over the tiles that are meant to be in full set
    standard_tile_names = [string for string in tile_names_in_storage_space_standard_formatted[:,0] if tile_name_standard_complete_array[2][0:27] in string] #check if a given tile has been annotated
    if len(standard_tile_names) == 0:
        counter_0 += 1
        unlabeled_array.append(tile_name_standard_complete_array)
    if len(standard_tile_names) == 1:
        counter_1 += 1
    if len(standard_tile_names) > 1:
        counter_gr1 += 1
        duplicated_standard_tile_names.append(tile_name_standard_complete_array)
print(counter_0, counter_1, counter_gr1)
unlabeled_array = np.array(unlabeled_array)

48 2345 46


In [ ]:
duplicated_standard_tile_names

In [ ]:
# Check whether unlabeled tiles are known to contain errors 

In [ ]:
counter_0 = 0
counter_geq1 = 0
unlabeled_array_no_known_errors = []

for tile_name in unlabeled_array: #iterate over the tiles that are meant to be in full set
    standard_tile_names_included_in_error_array = [string for string in tiles_errors[:,0] if tile_name[2][0:27] in string] #check if a given tile has been annotated
    if len(standard_tile_names_included_in_error_array) == 0:
        counter_0 += 1
        unlabeled_array_no_known_errors.append(tile_name)
        
    if len(standard_tile_names_included_in_error_array) >= 1:
        counter_geq1 += 1
print(counter_0, counter_geq1)

unlabeled_array_no_known_errors = np.array(unlabeled_array_no_known_errors)
np.save("unlabeled_tile_names_tile_urls.npy", unlabeled_array_no_known_errors[:,[0,1]])

In [ ]:
"""
# Determine which tiles included in the complete dataset array have not been labeled (tile names formatted)
counter_0 = 0
counter_1 = 0
counter_gr1 = 0
for tile_names_formatted in tile_names_in_storage_space_standard_formatted[:,1]: #iterate over the tiles that have actually been annotated
    strings_with_substring = [string for string in tile_names_tile_urls_complete_array_unique_formatted_tile_names[:,3] if tile_names_formatted in string] #check if a given tile is in the complete_array 
    if len(strings_with_substring) == 0:
        counter_0 += 1
    if len(strings_with_substring) == 1:
        counter_1 += 1
    if len(strings_with_substring) > 1:
        counter_gr1 += 1
print(counter_0, counter_1, counter_gr1)

# Check which formatted tile names that could be included in the dataset are missing
counter_0 = 0
counter_1 = 0
counter_gr1 = 0
for tile_name_formatted_complete_array in tile_names_tile_urls_complete_array_unique_formatted_tile_names: #iterate over the tiles that are meant to be in full set
    unlabeled = [string for string in tile_names_in_storage_space_standard_formatted[:,1] if tile_name_formatted_complete_array[3][0:46] in string] #check if a given tile has been annotated
    if len(unlabeled) == 0:
        counter_0 += 1
    if len(unlabeled) == 1:
        counter_1 += 1
    if len(unlabeled) > 1:
        counter_gr1 += 1
print(counter_0, counter_1, counter_gr1)
"""
# Determine which tiles included in the complete dataset array have not been labeled (standard naming convention)
## Use this to determine which tiles have not been annotated

# Download Tiles 